In [1]:
import json
import random
import pandas as pd

# 1.处理inter

In [2]:
name = "ml-100k"

inter_path = f"../../resource/{name}/{name}.inter"
user_path = f"../../resource/{name}/{name}.user"
item_path = f"../../resource/{name}/{name}.item"

w_pt_path = f"../../llm/{name}/{name}_pt.json"
w_ft_path = f"../../llm/{name}/{name}_ft.json"
w_pd_path = f"../../llm/{name}/{name}_pd.json"

In [3]:
# 读数据
inter_data = pd.read_csv(inter_path, sep="\t").sort_values(by=['user_id:token','rating:float', 'timestamp:float'], ascending=[True, False, True])
user_data = pd.read_csv(user_path, sep="\t").sort_values(by=['user_id:token'], ascending=[True])
item_data = pd.read_csv(item_path, sep="\t").sort_values(by=['item_id:token'], ascending=[True])
inter_data,user_data,item_data

(       user_id:token  item_id:token  rating:float  timestamp:float
 59972              1            168             5        874965478
 92487              1            172             5        874965478
 74577              1            165             5        874965518
 15764              1            196             5        874965677
 22971              1            166             5        874965677
 ...              ...            ...           ...              ...
 96489            943            585             1        888640250
 81005            943            450             1        888693158
 92536            943            227             1        888693158
 93377            943            449             1        888693158
 95003            943            230             1        888693158
 
 [100000 rows x 4 columns],
      user_id:token  age:token gender:token occupation:token zip_code:token
 0                1         24            M       technician          85711
 1

In [4]:
threshold = 3
max_length = 10
item_temp = "movie title: {movie_title}, release year: {release_year}, category: {class}, likes count: {likes_count}, dislikes count: {dislikes_count}, liking rate: {liking_rate}."
user_temp = "age: {age}, gender: {gender}, occupation: {occupation}, zipcode: {zip_code}."
# inter_temp = "The user user_{user_id} likes the following movies: {likes} and dislikes the following movies: {dislikes}."
user_train_temp = """You are a recommendation system that gives you a list of movies that the user likes and dislikes, and predicts whether the user will like the target movie or not. Returns only whether the user likes the target movie: Yes or No. # noqa: E501

User Information:
{user_info}

Likes List:
{likes_list}

Dislikes List:
{dislikes_list}

Target Movie:
{targe_movie}
"""

In [5]:
def build_item_other(row):
    df1 = inter_data[inter_data['item_id:token'] == row['item_id:token']]
    total = df1.shape[0]
    likes_count = df1[df1['rating:float'] > threshold].shape[0]
    dislikes_count = total - likes_count
    liking_rate = round(likes_count/total, 4)
    return {'likes_count':likes_count,'dislikes_count':dislikes_count,'liking_rate':liking_rate}

In [6]:
item_data['other'] = item_data.apply(lambda row: build_item_other(row), axis=1)
item_data['other'].iloc[0]

{'likes_count': 321, 'dislikes_count': 131, 'liking_rate': 0.7102}

In [7]:
# 拼接信息, json
item_data['info'] = item_data.apply(lambda row: {'text':item_temp.replace('{movie_title}',row["movie_title:token_seq"]).replace('{item_id}',str(row["item_id:token"])).replace('{release_year}',str(row["release_year:token"])).replace('{class}',row["class:token_seq"]).replace('{likes_count}',str(row['other']["likes_count"])).replace('{dislikes_count}',str(row['other']["dislikes_count"])).replace('{liking_rate}',str(row['other']["liking_rate"]))}, axis=1)
item_data['info'].iloc[0]

{'text': "movie title: Toy Story, release year: 1995, category: Animation Children's Comedy, likes count: 321, dislikes count: 131, liking rate: 0.7102."}

In [8]:
item_data.iloc[2]

item_id:token                                                            3
movie_title:token_seq                                           Four Rooms
release_year:token                                                    1995
class:token_seq                                                   Thriller
other                    {'likes_count': 34, 'dislikes_count': 56, 'lik...
info                     {'text': 'movie title: Four Rooms, release yea...
Name: 2, dtype: object

In [9]:
# 分组&聚合
data_new = inter_data
data_u = data_new.groupby(['user_id:token']).count()
data_u["count"] = data_u["item_id:token"]
# data_u = data_u.drop(['item_id:token','num_repeat:float','timestamp:float'], axis=1)

# 筛选=>10 core
data_u = data_u[data_u["count"] >= 10]

# 按user_id筛选
inter_data = data_new[data_new["user_id:token"].isin(data_u.index)]
# 按user_id分组，并将其他字段存为列表
inter_data = inter_data.groupby('user_id:token').agg(list).reset_index()
inter_data

,user_id:token,item_id:token,rating:float,timestamp:float
0,1,"[168, 172, 165, 196, 166, 14, 127, 181, 109, 1...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[874965478, 874965478, 874965518, 874965677, 8..."
1,2,"[275, 251, 50, 313, 242, 283, 311, 100, 127, 2...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, ...","[888550939, 888552084, 888552084, 888552084, 8..."
2,3,"[328, 321, 347, 340, 346, 320, 344, 342, 331, ...","[5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[889237455, 889237455, 889237455, 889237455, 8..."
3,4,"[258, 300, 303, 329, 327, 359, 362, 360, 361, ...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, ...","[892001374, 892001445, 892002352, 892002352, 8..."
4,5,"[257, 100, 109, 181, 89, 228, 172, 174, 396, 4...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[875635239, 875635349, 875635350, 875635757, 8..."
...,...,...,...,...
938,939,"[689, 326, 127, 9, 257, 1190, 742, 222, 591, 2...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[880260636, 880260636, 880260745, 880260745, 8..."
939,940,"[300, 258, 313, 482, 508, 96, 427, 709, 8, 56,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, ...","[884801316, 884801316, 884801316, 885921198, 8..."
940,941,"[117, 408, 181, 298, 919, 124, 1, 258, 300, 29...","[5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, ...","[875048886, 875048886, 875048887, 875048887, 8..."
941,942,"[347, 272, 304, 300, 500, 50, 282, 528, 71, 99...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[891282396, 891282420, 891282457, 891282564, 8..."


In [10]:
def cal_arr(ls,num):
    arr = []
    for i in range(0, len(ls), num):
        a1 = ls[i:i+num]
        if len(a1)==num:
            arr.append(a1)
    return arr
def build_user_inter(row):
    rs = row['rating:float']
    p_len = len([x for x in rs if x > threshold])
    p_rs = rs[:p_len]
    n_rs = rs[p_len:]

    ids = row['item_id:token']
    p_id_groups = ids[:p_len]
    n_id_groups = ids[p_len:]

    # 6p,4n
    p_num,n_num = 6,4
    p_rs1 = cal_arr(p_rs,p_num)
    n_rs1 = cal_arr(n_rs,n_num)

    p_id_groups1 = cal_arr(p_id_groups,p_num)
    n_id_groups1 = cal_arr(n_id_groups,n_num)
    
    # 对齐两个列表
    min_num = min(len(p_rs1), len(n_rs1))
    p_rs = p_rs1[:min_num]
    n_rs = n_rs1[:min_num]

    p_id_groups = p_id_groups1[:min_num]
    n_id_groups = n_id_groups1[:min_num]

    inter = {'p_id_groups': p_id_groups, 'n_id_groups': n_id_groups, 'p_rs': p_rs, 'n_rs': n_rs}
    return inter

inter_data['user_inter'] = inter_data.apply(lambda row: build_user_inter(row), axis=1)
inter_data.iloc[0]

user_id:token                                                      1
item_id:token      [168, 172, 165, 196, 166, 14, 127, 181, 109, 1...
rating:float       [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...
timestamp:float    [874965478, 874965478, 874965518, 874965677, 8...
user_inter         {'p_id_groups': [[168, 172, 165, 196, 166, 14]...
Name: 0, dtype: object

In [11]:
# 拼接信息, json
gender = {'M': 'male', 'F': 'female'}
user_data['info'] = user_data.apply(lambda row: {'text':user_temp.replace('{user_id}',str(row["user_id:token"])).replace('{age}',str(row["age:token"])).replace('{gender}',gender[row["gender:token"]]).replace('{occupation}',row["occupation:token"]).replace('{zip_code}',str(row["zip_code:token"]))}, axis=1)
user_data['info'].iloc[0]

{'text': 'age: 24, gender: male, occupation: technician, zipcode: 85711.'}

In [12]:
def cal_label(rating):
    return "Yes" if rating > threshold else "No"

In [13]:
# 使用 iterrows() 方法遍历每一行
for idx, row in inter_data.iterrows():
    ids = row['item_id:token']
    rs = row['rating:float']
    # 每10个取一组
    g_len = max_length+1
    ids_arr = [ids[i:i+g_len] for i in range(0, len(ids), g_len)]
    rs_arr = [rs[i:i+g_len] for i in range(0, len(rs), g_len)]
    for id_arr,r_arr in zip(ids_arr,rs_arr):
        if len(id_arr) < 10:
            break
        his = id_arr[len(id_arr)-1]
        t_id = id_arr[len(id_arr)-1]
        

In [14]:
inter_data

,user_id:token,item_id:token,rating:float,timestamp:float,user_inter
0,1,"[168, 172, 165, 196, 166, 14, 127, 181, 109, 1...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[874965478, 874965478, 874965518, 874965677, 8...","{'p_id_groups': [[168, 172, 165, 196, 166, 14]..."
1,2,"[275, 251, 50, 313, 242, 283, 311, 100, 127, 2...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, ...","[888550939, 888552084, 888552084, 888552084, 8...","{'p_id_groups': [[275, 251, 50, 313, 242, 283]..."
2,3,"[328, 321, 347, 340, 346, 320, 344, 342, 331, ...","[5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[889237455, 889237455, 889237455, 889237455, 8...","{'p_id_groups': [[328, 321, 347, 340, 346, 320..."
3,4,"[258, 300, 303, 329, 327, 359, 362, 360, 361, ...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, ...","[892001374, 892001445, 892002352, 892002352, 8...","{'p_id_groups': [[258, 300, 303, 329, 327, 359..."
4,5,"[257, 100, 109, 181, 89, 228, 172, 174, 396, 4...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[875635239, 875635349, 875635350, 875635757, 8...","{'p_id_groups': [[257, 100, 109, 181, 89, 228]..."
...,...,...,...,...,...
938,939,"[689, 326, 127, 9, 257, 1190, 742, 222, 591, 2...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[880260636, 880260636, 880260745, 880260745, 8...","{'p_id_groups': [[689, 326, 127, 9, 257, 1190]..."
939,940,"[300, 258, 313, 482, 508, 96, 427, 709, 8, 56,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, ...","[884801316, 884801316, 884801316, 885921198, 8...","{'p_id_groups': [[300, 258, 313, 482, 508, 96]..."
940,941,"[117, 408, 181, 298, 919, 124, 1, 258, 300, 29...","[5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, ...","[875048886, 875048886, 875048887, 875048887, 8...","{'p_id_groups': [[117, 408, 181, 298, 919, 124..."
941,942,"[347, 272, 304, 300, 500, 50, 282, 528, 71, 99...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[891282396, 891282420, 891282457, 891282564, 8...","{'p_id_groups': [[347, 272, 304, 300, 500, 50]..."


In [15]:
def build_item_infos(ids):
    item_infos = ''
    for i in ids:
        item_row = item_data[item_data["item_id:token"]==i].iloc[0]
        item_infos += f'movie title: {item_row["movie_title:token_seq"]}, release year: {item_row["release_year:token"]}, category: {item_row["class:token_seq"]} \n'
    return item_infos

def build_inter(row):
    user_row = user_data[user_data["user_id:token"]==row["user_id:token"]].iloc[0]
    user_info = f'age: {user_row["age:token"]}, gender: {gender[user_row["gender:token"]]}, occupation: {user_row["occupation:token"]}, zipcode: {user_row["zip_code:token"]}'

    user_inter = row['user_inter']
    p_id_groups = user_inter['p_id_groups']
    n_id_groups = user_inter['n_id_groups']
    for p_ids,n_ids in zip(p_id_groups,n_id_groups):
        # [0,1)根据0.6和0.4的比例进行线性转换
        if random.random() < 0.6:
            iid = p_ids.pop()
            flag = 'Yes'
        else:
            iid = n_ids.pop()
            flag = 'No'
        target_item_row = item_data[item_data["item_id:token"]==iid].iloc[0]
        target_movie = f'movie title: {target_item_row["movie_title:token_seq"]}, release year: {target_item_row["release_year:token"]}, category: {target_item_row["class:token_seq"]}, likes count: {target_item_row["other"]["likes_count"]}, dislikes count: {target_item_row["other"]["dislikes_count"]}, liking rate: {target_item_row["other"]["liking_rate"]}'
    
        user_temp = user_train_temp.replace('{user_info}', user_info).replace('{targe_movie}',target_movie).replace('{likes_list}', build_item_infos(p_ids)).replace('{dislikes_list}', build_item_infos(n_ids)).replace('\n\n\n','\n\n')
        user_dict = {'instruction': user_temp, 'input': '', 'output': flag}
        if random.random() < 0.8:
            ft_list.append(user_dict)
        else:
            pd_list.append(user_dict)
        

In [16]:
# 拼接信息
ft_list = [] # 微调
pd_list = [] # 预测
for idx, row in inter_data.iterrows():
    build_inter(row)

In [17]:
def export_json(data,export_path):
    df = pd.DataFrame(data)
    # 导出为格式化后的 JSON 文件
    df.to_json(export_path, orient='records')

In [18]:
export_json(item_data['info'].tolist()+user_data['info'].tolist(),w_pt_path)
export_json(ft_list,w_ft_path)
export_json(pd_list,w_pd_path)

In [19]:
# 读取 JSON 文件到 DataFrame
text_df = pd.read_json(w_pt_path)
text_df

,text
0,"movie title: Toy Story, release year: 1995, ca..."
1,"movie title: GoldenEye, release year: 1995, ca..."
2,"movie title: Four Rooms, release year: 1995, c..."
3,"movie title: Get Shorty, release year: 1995, c..."
4,"movie title: Copycat, release year: 1995, cate..."
...,...
2620,"age: 26, gender: female, occupation: student, ..."
2621,"age: 32, gender: male, occupation: administrat..."
2622,"age: 20, gender: male, occupation: student, zi..."
2623,"age: 48, gender: female, occupation: librarian..."
